In [0]:
df = spark.table("workspace.default.ecommerce_data")
df.show()

+--------+----------+-----------+-------------+----------------+--------------------+-----+--------+--------------+---------+
|order_id|order_date|customer_id|customer_name|product_category|        product_name|price|quantity|payment_method|     city|
+--------+----------+-----------+-------------+----------------+--------------------+-----+--------+--------------+---------+
|    1001|2024-01-05|       C001|  Amit Sharma|     Electronics|      Wireless Mouse|  799|       1|           UPI|Bangalore|
|    1002|2024-01-06|       C002|  Priya Verma|         Fashion|        Cotton Kurti| 1299|       2|   Credit Card|    Delhi|
|    1003|2024-01-06|       C003|  Rahul Mehta|     Electronics|Bluetooth Headphones| 2499|       1|    Debit Card|   Mumbai|
|    1004|2024-01-07|       C004| Ananya Singh|  Home & Kitchen|     Electric Kettle| 1999|       1|           UPI|Hyderabad|
|    1005|2024-01-08|       C001|  Amit Sharma|     Electronics|       USB-C Charger|  999|       2|           UPI|Ban

# Create Logical Tables (from same data)

**Orders table**

In [0]:
orders_df = df.select(
    "order_id", "order_date", "customer_id",
    "product_name", "price", "quantity"
)
orders_df.show()

+--------+----------+-----------+--------------------+-----+--------+
|order_id|order_date|customer_id|        product_name|price|quantity|
+--------+----------+-----------+--------------------+-----+--------+
|    1001|2024-01-05|       C001|      Wireless Mouse|  799|       1|
|    1002|2024-01-06|       C002|        Cotton Kurti| 1299|       2|
|    1003|2024-01-06|       C003|Bluetooth Headphones| 2499|       1|
|    1004|2024-01-07|       C004|     Electric Kettle| 1999|       1|
|    1005|2024-01-08|       C001|       USB-C Charger|  999|       2|
|    1006|2024-01-09|       C005|          Face Serum| 1599|       1|
|    1007|2024-01-10|       C006|       Running Shoes| 3499|       1|
|    1008|2024-01-10|       C002|          Dinner Set| 2999|       1|
|    1009|2024-01-11|       C007|         Smart Watch| 4999|       1|
|    1010|2024-01-12|       C003|          Hair Dryer| 1899|       1|
+--------+----------+-----------+--------------------+-----+--------+



**Customers table**

In [0]:
customers_df = df.select(
    "customer_id", "customer_name", "city"
).dropDuplicates()
customers_df.show()

+-----------+-------------+---------+
|customer_id|customer_name|     city|
+-----------+-------------+---------+
|       C001|  Amit Sharma|Bangalore|
|       C002|  Priya Verma|    Delhi|
|       C003|  Rahul Mehta|   Mumbai|
|       C004| Ananya Singh|Hyderabad|
|       C005|   Sneha Iyer|  Chennai|
|       C006|  Vikas Gupta|     Pune|
|       C007|  Neha Kapoor|    Noida|
+-----------+-------------+---------+



**Payments table**

In [0]:
payments_df = df.select(
    "order_id", "payment_method"
)
payments_df.show()

+--------+--------------+
|order_id|payment_method|
+--------+--------------+
|    1001|           UPI|
|    1002|   Credit Card|
|    1003|    Debit Card|
|    1004|           UPI|
|    1005|           UPI|
|    1006|   Net Banking|
|    1007|   Credit Card|
|    1008|    Debit Card|
|    1009|           UPI|
|    1010|   Credit Card|
+--------+--------------+



# 🔗 COMPLEX JOIN QUERIES

**INNER JOIN - Orders + Customers**

In [0]:
orders_customers = orders_df.join(
    customers_df,
    on="customer_id",
    how="inner"
)

orders_customers.show()

+-----------+--------+----------+--------------------+-----+--------+-------------+---------+
|customer_id|order_id|order_date|        product_name|price|quantity|customer_name|     city|
+-----------+--------+----------+--------------------+-----+--------+-------------+---------+
|       C004|    1004|2024-01-07|     Electric Kettle| 1999|       1| Ananya Singh|Hyderabad|
|       C006|    1007|2024-01-10|       Running Shoes| 3499|       1|  Vikas Gupta|     Pune|
|       C007|    1009|2024-01-11|         Smart Watch| 4999|       1|  Neha Kapoor|    Noida|
|       C005|    1006|2024-01-09|          Face Serum| 1599|       1|   Sneha Iyer|  Chennai|
|       C001|    1005|2024-01-08|       USB-C Charger|  999|       2|  Amit Sharma|Bangalore|
|       C002|    1008|2024-01-10|          Dinner Set| 2999|       1|  Priya Verma|    Delhi|
|       C003|    1010|2024-01-12|          Hair Dryer| 1899|       1|  Rahul Mehta|   Mumbai|
|       C001|    1001|2024-01-05|      Wireless Mouse|  799|

**LEFT JOIN – Orders + Payments**

In [0]:
orders_payments = orders_df.join(
    payments_df,
    on="order_id",
    how="left"
)

orders_payments.show()

+--------+----------+-----------+--------------------+-----+--------+--------------+
|order_id|order_date|customer_id|        product_name|price|quantity|payment_method|
+--------+----------+-----------+--------------------+-----+--------+--------------+
|    1001|2024-01-05|       C001|      Wireless Mouse|  799|       1|           UPI|
|    1002|2024-01-06|       C002|        Cotton Kurti| 1299|       2|   Credit Card|
|    1003|2024-01-06|       C003|Bluetooth Headphones| 2499|       1|    Debit Card|
|    1004|2024-01-07|       C004|     Electric Kettle| 1999|       1|           UPI|
|    1005|2024-01-08|       C001|       USB-C Charger|  999|       2|           UPI|
|    1006|2024-01-09|       C005|          Face Serum| 1599|       1|   Net Banking|
|    1007|2024-01-10|       C006|       Running Shoes| 3499|       1|   Credit Card|
|    1008|2024-01-10|       C002|          Dinner Set| 2999|       1|    Debit Card|
|    1009|2024-01-11|       C007|         Smart Watch| 4999|     

**MULTI-TABLE JOIN (Orders + Customers + Payments)**

In [0]:
full_df = orders_df \
    .join(customers_df, "customer_id", "inner") \
    .join(payments_df, "order_id", "left")

full_df.show()

+--------+-----------+----------+--------------------+-----+--------+-------------+---------+--------------+
|order_id|customer_id|order_date|        product_name|price|quantity|customer_name|     city|payment_method|
+--------+-----------+----------+--------------------+-----+--------+-------------+---------+--------------+
|    1004|       C004|2024-01-07|     Electric Kettle| 1999|       1| Ananya Singh|Hyderabad|           UPI|
|    1007|       C006|2024-01-10|       Running Shoes| 3499|       1|  Vikas Gupta|     Pune|   Credit Card|
|    1009|       C007|2024-01-11|         Smart Watch| 4999|       1|  Neha Kapoor|    Noida|           UPI|
|    1006|       C005|2024-01-09|          Face Serum| 1599|       1|   Sneha Iyer|  Chennai|   Net Banking|
|    1005|       C001|2024-01-08|       USB-C Charger|  999|       2|  Amit Sharma|Bangalore|           UPI|
|    1008|       C002|2024-01-10|          Dinner Set| 2999|       1|  Priya Verma|    Delhi|    Debit Card|
|    1010|       C0

# Window Functions

**Running total of spend per customer**

In [0]:
# from pyspark.sql import functions as F
# from pyspark.sql.window import Window

window = Window.partitionBy("customer_id") \
               .orderBy("order_date") \
               .rowsBetween(Window.unboundedPreceding, Window.currentRow)

df = df.withColumn("order_value", F.col("price") * F.col("quantity"))
df = df.withColumn("running_total_spend", F.sum("order_value").over(window))
display(df.select("customer_id", "order_date", "order_value", "running_total_spend"))

+-----------+----------+-----------+-------------------+
|customer_id|order_date|order_value|running_total_spend|
+-----------+----------+-----------+-------------------+
|       C001|2024-01-05|        799|                799|
|       C001|2024-01-08|       1998|               2797|
|       C002|2024-01-06|       2598|               2598|
|       C002|2024-01-10|       2999|               5597|
|       C003|2024-01-06|       2499|               2499|
|       C003|2024-01-12|       1899|               4398|
|       C004|2024-01-07|       1999|               1999|
|       C005|2024-01-09|       1599|               1599|
|       C006|2024-01-10|       3499|               3499|
|       C007|2024-01-11|       4999|               4999|
+-----------+----------+-----------+-------------------+



**Rank products by total revenue**

In [0]:
window = Window.orderBy(F.desc("revenue"))

df.withColumn("order_value", F.col("price") * F.col("quantity")) \
  .groupBy("product_name") \
  .agg(F.sum("order_value").alias("revenue")) \
  .withColumn("product_rank", F.rank().over(window)) \
  .show()

/databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/expressions.py:1134: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


+--------------------+-------+------------+
|        product_name|revenue|product_rank|
+--------------------+-------+------------+
|         Smart Watch|   4999|           1|
|       Running Shoes|   3499|           2|
|          Dinner Set|   2999|           3|
|        Cotton Kurti|   2598|           4|
|Bluetooth Headphones|   2499|           5|
|     Electric Kettle|   1999|           6|
|       USB-C Charger|   1998|           7|
|          Hair Dryer|   1899|           8|
|          Face Serum|   1599|           9|
|      Wireless Mouse|    799|          10|
+--------------------+-------+------------+



**Average order value per city (derived metric)**

In [0]:
df.withColumn("order_value", F.col("price") * F.col("quantity")) \
  .groupBy("city") \
  .agg(F.avg("order_value").alias("avg_order_value")) \
  .orderBy(F.desc("avg_order_value")) \
  .show()


+---------+---------------+
|     city|avg_order_value|
+---------+---------------+
|    Noida|         4999.0|
|     Pune|         3499.0|
|    Delhi|         2798.5|
|   Mumbai|         2199.0|
|Hyderabad|         1999.0|
|  Chennai|         1599.0|
|Bangalore|         1398.5|
+---------+---------------+

